import sys
print(sys.executable)


!{sys.executable} -m pip install newspaper3k


!{sys.executable} -m pip install 'lxml[html_clean]'


We use newspaper3k to extract text from web pages.

In [19]:
from newspaper import Article
import pandas as pd

print("✅ All libraries loaded.")


✅ All libraries loaded.


In [80]:
import os
from datetime import datetime

# Create folder to save articles
output_folder = "Corpus_deepseek"
os.makedirs(output_folder, exist_ok=True)

# Helper function to detect source from URL
def get_source_from_url(url):
    if "theatlantic.com" in url:
        return "The Atlantic"
    if "ft.com" in url:
        return "Financial Times"
    if "aljazeera.com" in url:
        return "Al Jazeera"
    if "scmp.com" in url:
        return "South China Morning Post"
    if "bbc.com" in url:
        return "BBC"
    if "nytimes.com" in url:
        return "THe New York Times"
    if "cnn.com" in url:
        return "CNN"
    if "theguardian.com" in url:
        return "The Guardian"
    if "dw.com" in url:
        return "DW"
    elif "newyorker.com" in url:
        return "The New Yorker"
    else:
        return "Unknown"

# Add some article URLs 
urls = [
    # The Atlantic
    "https://www.theatlantic.com/technology/archive/2025/01/deepseek-china-ai/681481/",
    "https://www.theatlantic.com/newsletters/archive/2025/02/what-is-ai-distillation/681616/",
    "https://www.theatlantic.com/newsletters/archive/2025/01/the-deepseek-wake-up-call/681512/",
    "https://www.theatlantic.com/ideas/archive/2025/01/deepseek-ai-investment-tech/681516/",
    # The New Yorker
    "https://www.newyorker.com/news/the-financial-page/is-deepseek-chinas-sputnik-moment",
    # FT(has paying wall)
    "https://www.ft.com/content/9f5cbee8-b09c-4274-bda9-7245ca97352e",
    "https://www.ft.com/content/0c497603-0f97-4b85-a52f-019e24329a75",
    "https://www.ft.com/content/fb5c11bb-1d4b-465f-8283-451a19a3d425",
    "https://www.ft.com/content/04512d41-70a7-4b53-8b50-0c9d28c7b80e",
    "https://www.ft.com/content/5684fb1f-1a84-4542-8fe9-2fcae9653f87",
    "https://www.ft.com/content/c82933fe-be28-463b-8336-d71a2ff5bbbf",
    "https://www.ft.com/content/837a9ac7-be8b-476f-bf66-f03443caef11",
    # Al Jazeera
    "https://www.aljazeera.com/economy/2025/1/28/why-chinas-ai-startup-deepseek-is-sending-shockwaves-through-global-tech",
    "https://www.aljazeera.com/news/2025/1/29/ai-game-changer-or-overhyped-deepseek-faces-scrutiny-over-bold-claims",
    "https://www.aljazeera.com/news/2025/2/6/which-countries-have-banned-deepseek-and-why",
    "https://www.aljazeera.com/economy/2025/4/24/deepseek-transferred-data-without-consent-south-korean-watchdog-says",
    "https://www.aljazeera.com/news/2025/2/17/south-korea-removes-deepseek-from-app-stores-pending-privacy-review",
    "https://www.aljazeera.com/economy/2025/1/27/chinas-deepseek-causes-rout-among-ai-linked-stocks",
    # South China Morning Post
    "https://www.scmp.com/tech/tech-trends/article/3292507/chinese-start-deepseek-launches-ai-model-outperforms-meta-openai-products",
    "https://www.scmp.com/tech/big-tech/article/3292447/tech-war-2024-china-catches-fast-ai-race-us-chip-curbs-cast-shadow",
    "https://www.scmp.com/tech/tech-trends/article/3293050/meet-deepseek-chinese-start-changing-how-ai-models-are-trained",
    "https://www.scmp.com/tech/big-tech/article/3294357/chinas-ai-disrupter-deepseek-bets-low-key-team-young-geniuses-beat-us-giants",
    "https://www.scmp.com/tech/big-tech/article/3296018/tech-war-china-poised-expand-ai-infrastructure-keep-pace-us-stargate-project",
    "https://www.scmp.com/tech/tech-trends/article/3296406/chinese-ai-disrupter-deepseek-claims-top-spot-us-app-store-dethroning-chatgpt",
    "https://www.scmp.com/news/china/article/3296510/us-tech-stocks-plunge-china-ais-unexpectedly-strong-showing",
    "https://www.scmp.com/opinion/china-opinion/article/3296426/chinese-start-deepseek-looks-set-rewrite-rules-ai-game",
    "https://www.scmp.com/business/china-business/article/3296531/hong-kong-stocks-gain-valuation-focus-deepseek-sparks-us-tech-rout",
    "https://www.scmp.com/tech/tech-trends/article/3296503/deepseeks-tech-breakthrough-hailed-china-answer-win-ai-war",
    "https://www.scmp.com/business/china-business/article/3296562/deepseek-breakthrough-raises-investor-concerns-over-need-massive-ai-spending",
    "https://www.scmp.com/news/world/united-states-canada/article/3296575/deepseek-shows-china-only-6-months-behind-us-state-art-ai-developers-say",
    "https://www.scmp.com/tech/tech-trends/article/3296606/chinese-ai-start-deepseek-breaks-lunar-new-year-its-success-rocks-wall-street",
    "https://www.scmp.com/tech/tech-war/article/3296656/white-house-evaluates-effect-china-ai-app-deepseek-national-security",
    "https://www.scmp.com/news/world/united-states-canada/article/3296667/microsoft-openai-investigate-chinas-deepseek-over-data-breach",
    "https://www.scmp.com/news/china/article/3296743/trumps-commerce-secretary-pick-vows-keep-china-using-us-tech-compete-us",
    # BBC
    "https://www.bbc.com/news/articles/c5yv5976z9po",
    "https://www.bbc.com/future/article/20250131-what-does-deepseeks-new-app-mean-for-the-future-of-ai",
    "https://www.bbc.com/news/articles/c8d95v0nr1yo",
    "https://www.bbc.com/news/articles/cy7g45g2nxno",
    "https://www.bbc.com/news/articles/clyzym0vn8go",
    "https://www.bbc.com/news/articles/c4gex0x87g4o",
    "https://www.bbc.com/news/articles/c0qw7z2v1pgo",
    "https://www.bbc.com/news/articles/c9w5d9new0yo",
    "https://www.bbc.com/news/articles/cqx9zn27700o",
    "https://www.bbc.com/news/articles/cx2k7r5nrvpo",
    "https://www.bbc.com/news/articles/cp8qglr9r74o",
    "https://www.bbc.com/news/articles/c17qqgwdk78o",
    "https://www.bbc.com/audio/play/w3ct681k",
    "https://www.bbc.com/news/articles/c4gpq01rvd4o",
    "https://www.bbc.com/news/articles/c9vm1m8wpr9o",
    "https://www.bbc.com/news/articles/cd643wx888qo",
    "https://www.bbc.com/news/videos/c77rd5ddxn8o",
    "https://www.bbc.com/news/articles/cx2jxvn0r51o",
    "https://www.bbc.com/news/articles/czepw096wy4o",
    "https://www.bbc.com/news/articles/c0rq0vyd549o",
    "https://www.bbc.com/news/articles/c17evw0k8g8o",
    # The New York Times
    "https://www.nytimes.com/2025/01/27/technology/what-is-deepseek-china-ai.html",
    "https://www.nytimes.com/2025/01/27/technology/deepseek-ai-chatbot-first-impressions.html",
    "https://www.nytimes.com/2025/01/28/technology/china-deepseek-ai-silicon-valley.html#:~:text=But%20DeepSeek's%20results%20show%20that,in%20a%20matter%20of%20weeks.",
    "https://www.nytimes.com/2025/01/28/business/deepseek-china-reaction.html",
    "https://www.nytimes.com/2025/01/29/business/deepseek-china-liang-wenfeng.html",
    "https://www.nytimes.com/2025/01/29/technology/meta-deepseek-ai-open-source.html",
    "https://www.nytimes.com/2025/01/29/technology/openai-deepseek-data-harvest.html",
    "https://www.nytimes.com/2025/01/29/technology/deepseek-ai-startups-venture-capital.html",
    "https://www.nytimes.com/2025/01/31/technology/deepseek-chinese-propaganda.html",
    "https://www.nytimes.com/2025/02/02/world/asia/deepseek-china-ai-censorship.html",
    "https://www.nytimes.com/2025/02/03/podcasts/the-daily/china-ai-deepseek.html",
    "https://www.nytimes.com/2025/02/04/opinion/deepseek-ai-big-tech.html",
    "https://www.nytimes.com/2025/02/07/business/ai-deepseek-nvidia-tesla.html",
    "https://www.nytimes.com/2025/02/08/technology/deepseek-data-centers-ai.html",
    "https://www.nytimes.com/2025/02/10/world/asia/china-deepseek-education.html",
    "https://www.nytimes.com/2025/02/12/technology/deepseek-ai-chip-costs.html",
    "https://www.nytimes.com/2025/02/14/business/deepseek-openai-jevons-paradox.html",
    "https://www.nytimes.com/2025/02/17/business/south-korea-deepseek-china-ai.html",
    # CNN
    "https://www.cnn.com/2025/01/27/tech/deepseek-stocks-ai-china",
    "https://www.cnn.com/2025/01/27/tech/deepseek-ai-explainer",
    "https://www.cnn.com/2025/01/28/business/deepseek-ai-nvidia-nightcap",
    "https://www.cnn.com/2025/01/28/tech/deepseek-trump-ai",
    "https://www.cnn.com/2025/01/28/china/china-deepseek-ai-success-tech-intl-hnk",
    "https://www.cnn.com/2025/01/29/china/deepseek-ai-china-censorship-moderation-intl-hnk",
    "https://www.cnn.com/2025/01/29/tech/deepseek-silicon-valley-ai",
    "https://www.cnn.com/2025/02/04/business/australia-deepseek-ban-security-concerns-intl",
    "https://www.cnn.com/2025/02/06/tech/deepseek-ai-us-ban-bill",
    "https://www.cnn.com/2025/02/10/tech/china-deepseek-ai-disruption-hnk-intl",
    "https://www.cnn.com/2025/02/14/tech/deepseek-ai-openai-hype#:~:text=What%20makes%20DeepSeek%20significant%20is,process%20as%20it%20answers%20questions.",
    "https://www.cnn.com/2025/02/14/tech/china-baidu-deepseek-ai-competition-free-services-intl-hnk",
    # The Guardian
    "https://www.theguardian.com/technology/2025/jan/28/we-tried-out-deepseek-it-works-well-until-we-asked-it-about-tiananmen-square-and-taiwan",
    "https://www.theguardian.com/technology/2025/jan/28/who-is-behind-deepseek-and-how-did-it-achieve-its-ai-sputnik-moment",
    "https://www.theguardian.com/technology/deepseek",
    "https://www.theguardian.com/business/2025/jan/27/what-is-deepseek-and-why-did-us-tech-stocks-fall",
    "https://www.theguardian.com/technology/2025/feb/03/deepseek-chinese-ai-company-competitors-chatgpt",
    "https://www.theguardian.com/commentisfree/2025/feb/02/deepseek-ai-veil-of-mystique-tech-bros-fear",
    "https://www.theguardian.com/technology/2025/jan/27/deepseek-cyberattack-ai",
    "https://www.theguardian.com/technology/2025/jan/29/deepseek-artificial-intelligence-ai-safety-risk-yoshua-bengio",
    "https://www.theguardian.com/technology/2025/feb/04/deepseek-banned-from-australian-government-devices-over-national-security-concerns",
    "https://www.theguardian.com/technology/2025/feb/01/deepseek-chatgpt-grok-gemini-claude-meta-ai-which-is-the-best-ai-assistant-we-put-them-to-the-test",
    "https://www.theguardian.com/technology/2025/feb/01/was-this-the-week-deepseek-started-the-slow-unwinding-of-the-ai-bet",
    "https://www.theguardian.com/technology/2025/jan/28/donald-trump-china-deepseek-ai-chatbot-shares",
    "https://www.theguardian.com/technology/video/2025/jan/28/how-chatgpt-gemini-and-deepseek-deal-with-tiananmen-square-and-taiwan-questions-video",
    "https://www.theguardian.com/technology/2025/jan/29/openai-chatgpt-deepseek-china-us-ai-models",
    "https://www.theguardian.com/business/2025/jan/27/tech-shares-asia-europe-fall-china-ai-deepseek",
    "https://www.theguardian.com/news/2025/feb/05/diving-into-deepseek-inside-the-7-february-guardian-weekly",
    "https://www.theguardian.com/technology/2025/jan/29/deepseek-blocked-some-app-stores-italy-questions-data-use",
    "https://www.theguardian.com/technology/2025/feb/01/ai-deepseek-cheap-china-google-apple",
    "https://www.theguardian.com/technology/2025/feb/02/as-deepseek-upends-the-ai-industry-one-group-is-urging-australia-to-embrace-the-opportunity",
    "https://www.theguardian.com/commentisfree/2025/jan/31/tech-bros-deepseek-china-sam-altman-openai",
    "https://www.theguardian.com/technology/2025/jan/28/deepseek-openai-elon-musk-don-lemon-crypto-kalshi",
    "https://www.theguardian.com/us-news/2025/jan/28/first-thing-donald-trump-calls-chinas-deepseek-ai-chatbot-a-wake-up-call",
    "https://www.theguardian.com/commentisfree/2025/jan/28/deepseek-r1-ai-world-chinese-chatbot-tech-world-western",
    "https://www.theguardian.com/technology/2025/jan/28/deepseek-ai-us-response",
    "https://www.theguardian.com/technology/2025/jan/28/deepseek-selloff",
    "https://www.theguardian.com/technology/2025/jan/28/experts-urge-caution-over-use-of-chinese-ai-deepseek",
    "https://www.theguardian.com/technology/2025/feb/26/nvidia-fourth-quarter-earnings-deepseek",
    # DW
    "https://www.dw.com/en/what-does-chinas-deepseek-mean-for-ais-energy-and-water-use/a-71459557",
    "https://www.dw.com/en/what-questions-will-chinas-deepseek-not-answer/a-71470843",
    "https://www.dw.com/en/why-is-chinas-deepseek-sending-ai-stocks-spinning/a-71427330",
    "https://www.dw.com/en/deepseek-cheap-artificial-intelligence-model-shakes-up-tech-industry/video-71436353",
    "https://www.dw.com/en/trump-deepseek-ai-competition/a-71430459",
    "https://www.dw.com/en/what-deepseeks-ai-revolution-means-for-you/video-71458752",
    "https://www.dw.com/en/south-korea-temporarily-bans-deepseek-ai-app/a-71634702",
    "https://www.dw.com/embed/480/av-71436353",
    
]
    
metadata = []

for url in urls:
    try:
        article = Article(url)
        article.download()
        article.parse()

        title = article.title.strip().replace('\n', ' ')
        author = ', '.join(article.authors) if article.authors else "Unknown"
        date = article.publish_date.strftime("%Y-%m-%d") if article.publish_date else "Unknown"
        text = article.text.strip()
        source = get_source_from_url(url)

        filename = title.replace(" ", "_")[:50] + ".txt"
        filepath = os.path.join(output_folder, filename)
        
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(text)

        metadata.append({
            "title": title,
            "author": author,
            "date": date,
            "source": source,
            "url": url,
            "filename": filename
        })

        print(f"✅ Saved: {title}")

    except Exception as e:
        print(f"❌ Error processing {url}: {e}")

# Save metadata to CSV
df = pd.DataFrame(metadata)
df.to_csv(os.path.join(output_folder, "metadata_all_texts.csv"), index=False)

print("📁 Done! Articles and metadata saved.")


✅ Saved: China’s DeepSeek Surprise
✅ Saved: What Is AI Distillation?
✅ Saved: The DeepSeek Wake-Up Call
✅ Saved: DeepSeek’s Chatbot Has an Important Message
✅ Saved: Is DeepSeek China’s Sputnik Moment?
✅ Saved: Global AI rivalry is a dangerous game
✅ Saved: DeepSeek is decoding Chinese destiny
✅ Saved: DeepSeek focuses on research over revenue in contrast to Silicon Valley
✅ Saved: US House panel probes whether DeepSeek used restricted Nvidia chips
✅ Saved: DeepSeek spreads across China with Beijing’s backing
✅ Saved: Here’s what the sellside is saying about DeepSeek
✅ Saved: The story behind DeepSeek’s breakthrough
✅ Saved: What’s DeepSeek, China’s AI startup sending shockwaves through global tech?
✅ Saved: China’s DeepSeek faces questions over claims after shaking up global tech
✅ Saved: Which countries have banned DeepSeek and why?
✅ Saved: DeepSeek transferred data without consent, South Korean watchdog says
✅ Saved: South Korea removes DeepSeek from app stores pending privacy revi

In [91]:
import os

# Specify the directory
folder_path = "./Corpus_deepseek/"

# List to store the article texts
all_texts = []

# Loop over files directly in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # Only read .txt files
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                all_texts.append(file.read())
        except UnicodeDecodeError:
            try:
                with open(file_path, 'r', encoding='latin-1') as file:
                    all_texts.append(file.read())
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

# Join all texts into one string
all_texts_str = " ".join(all_texts)
print(all_texts_str[:1000])  # Preview first 1000 characters
print(type(all_texts_str))

# Write combined text to file
with open("./all_texts.txt", mode="w", encoding="utf-8") as file:
    file.write(all_texts_str)

print("Combined text has been written to 'all_texts.txt'")


Try unlimited access Only Skr10 for 4 weeks

Then Skr739 per month. Complete digital access to quality FT journalism on any device. Cancel anytime during your trial. CNN —

A surprisingly efficient and powerful Chinese AI model has taken the technology industry by storm. It’s called DeepSeek R1, and it’s rattling nerves on Wall Street.

The new AI model was developed by DeepSeek, a startup that was born just a year ago and has somehow managed a breakthrough that famed tech investor Marc Andreessen has called “AI’s Sputnik moment”: R1 can nearly match the capabilities of its far more famous rivals, including OpenAI’s GPT-4, Meta’s Llama and Google’s Gemini — but at a fraction of the cost.

The company said it had spent just $5.6 million powering its base AI model, compared with the hundreds of millions, if not billions of dollars US companies spend on their AI technologies. That’s even more shocking when considering that the United States has worked for years to restrict the supply of h